In [213]:
import sys, time
import pandas as pd
import datetime as dt
from IPython.display import display
import numpy as np

In [214]:
! pip install -U airportsdata

In [226]:
airports['KJFK']

{'icao': 'KJFK',
 'iata': 'JFK',
 'name': 'John F Kennedy International Airport',
 'city': 'New York',
 'subd': 'New York',
 'country': 'US',
 'elevation': 13.0,
 'lat': 40.6399277777778,
 'lon': -73.7786925,
 'tz': 'America/New_York',
 'lid': 'JFK'}

In [215]:
import airportsdata
airports = airportsdata.load()
# Replace non-ascii characters
for (k,v) in airports.items():
    old = airports[k]['name']
    airports[k]['name'] = ''.join([i if ord(i) < 128 else ' ' for i in old])

In [216]:
path = 'flightlist_20191201_20191231.csv'

In [217]:
#helpers
space = " "
tab = space * 2
two_tabs = space * 4
nl = "\n"
left_bracket = f"{'['}"
right_bracket = f"{']'}"

In [218]:
#Write an edge
def write_edge(source,target,value,f):
    record = (f"{tab}edge{nl}"
              f"{tab}{left_bracket}{nl}"
              f"{two_tabs}source{space}\"{source}\"{nl}"
              f"{two_tabs}target{space}\"{target}\"{nl}"
              f"{two_tabs}value{space}{value}{nl}"
              f"{tab}{right_bracket}{nl}")
    f.write(record.encode('ascii'))

In [240]:
#Write a node
def write_node(airport,f):
    record = (f"{tab}node{nl}"
              f"{tab}{left_bracket}{nl}"
              f"{two_tabs}id{space}\"{airport['icao']}\"{nl}"
              f"{two_tabs}label{space}\"{airport['icao']}\"{nl}"
              f"{two_tabs}data{space}\"{airport['name']}-{airport['country']}\"{nl}"
              f"{tab}{right_bracket}{nl}")
    f.write(record.encode('ascii'))

In [227]:
def csv_to_gml(csv_file_path):
    df = pd.read_csv(csv_file_path,usecols=['origin','destination'],dtype=str)
    df = df.dropna()
    df = df.groupby(['origin','destination'], as_index=False).size().rename(columns={"size":"flight_count"})
    f = open(f"{csv_file_path.split('.')[0]}.gml", "wb")
    #loop helpers
    added = []
    added_edges_sets = []
    ind = 0

    #Root node
    f.write(f"graph{nl}".encode("ascii"))
    f.write(f"{left_bracket}{nl}".encode("ascii"))

    #Generate nodes
    for i, r in df.iterrows():
        #increment, as index not reliable
        ind += 1
        origin = r['origin']
        destination = r['destination']
        #Check for duplicates
        
        # Also we want only airport with iata code, these airport are tend not to be private or military airports
        if (origin not in added and origin in airports and airports[origin]['iata'] != ''):
            #Add to list
            added.append(origin)
            write_node(airports[origin],f)
        if (destination not in added and destination in airports and airports[destination]['iata'] != ''):
            #Add to list
            added.append(destination)
            write_node(airports[destination],f)
        #print the progress
        if ind % 100000 == 0:
            print(f"Writing Nodes {ind} of {len(df)}")

    print(nl+"Printing nodes over")

    #flush index
    ind = 0    
    #Generate edges    
    for i, r in df.iterrows():
        #increment, as index not reliable
        ind += 1
        origin = r['origin']
        destination = r['destination']
        edge_set = set([origin,destination])
        if (origin in added and destination in added and edge_set not in added_edges_sets):
            #Find all the records between origin and destination
            cond = ((df['origin'] == origin) & (df['destination'] == destination))|((df['origin'] == destination) & (df['destination'] == origin))
            n_flights = df.loc[cond]['flight_count'].sum()
            write_edge(origin, destination, n_flights,f)
            added_edges_sets.append(edge_set)
        if ind % 1000 == 0:
            sys.stdout.flush()
            sys.stdout.write('\r')
            sys.stdout.flush()
            sys.stdout.write(f"Writing Edges {ind} of {len(df)}")


    print(nl+"Printing nodes and edges over")

    #closing node
    f.write(f"{right_bracket}{nl}".encode("ascii"))
    f.close()

In [245]:
csv_to_gml(path)

Writing Nodes 100000 of 170990

Printing nodes over
Writing Edges 170000 of 170990
Printing nodes and edges over


In [246]:
csv_to_gml("flightlist_20200201_20200229.csv")

Writing Nodes 100000 of 181146

Printing nodes over
Writing Edges 181000 of 181146
Printing nodes and edges over


In [241]:
csv_to_gml("flightlist_20200401_20200430.csv")

Writing Nodes 100000 of 114568

Printing nodes over
Writing Edges 114000 of 114568
Printing nodes and edges over


In [248]:
import networkx as nx
g = nx.read_gml("flightlist_20191201_20191231.gml")

In [251]:
airports['VIAG']

{'icao': 'VIAG',
 'iata': 'AGR',
 'name': 'Agra Airport',
 'city': '',
 'subd': 'Uttar-Pradesh',
 'country': 'IN',
 'elevation': 551.0,
 'lat': 27.1557998657,
 'lon': 77.960899353,
 'tz': 'Asia/Kolkata',
 'lid': ''}

In [252]:
308/1590

0.19371069182389938

In [257]:
airports['OMDW']

{'icao': 'OMDW',
 'iata': 'DWC',
 'name': 'Al Maktoum International Airport',
 'city': 'Jebel Ali',
 'subd': 'Dubai',
 'country': 'AE',
 'elevation': 114.0,
 'lat': 24.8966666667,
 'lon': 55.1613888889,
 'tz': 'Asia/Dubai',
 'lid': ''}